# Volumeter

TODO
- measurementCycle(loweringSpeed)
- calibrateHolder()
- syncMeasurementData()
- calculateVolume()
- visualize()
- exportCSV()

In [155]:
import serial
import pandas as pd
import numpy as np
import time
import os

In [150]:
global loadCellConstants
global mcSerial
global sensorSerial

def readLoadCellCalibration():
    cal = np.genfromtxt('loadCellCalibration.csv', delimiter=',')
    return cal

def loadCellToGrams(value):
    return value*loadCellConstants[0] + loadCellConstants[1]

def initMotionControl():
    global mcSerial
    motionControlPort = '/dev/ttyUSB1'
    mcSerial = serial.Serial(port=motionControlPort, baudrate=115200)

def initSensors():
    global sensorSerial
    global loadCellConstants
    loadCellConstants = readLoadCellCalibration()
    
    sensorPort = '/dev/ttyUSB0'
    sensorSerial = serial.Serial(port=sensorPort, baudrate=115200)

def closeSerial(serial):
    serial.close()

def sendMotionCommand(cmd):
    mcSerial.write('{0}\n'.format(cmd).encode('utf-8'))
    mcSerial.flushInput()
    grbl_out = mcSerial.readline()
    return grbl_out.decode()

def motionFinished(wait=False):
    finished = False
    if(wait):
        while(not finished):
            time.sleep(0.5)
            grbl_out = sendMotionCommand('?')
            finished = 'Idle' in str(grbl_out)
        return True
    else:
        grbl_out = sendMotionCommand(serial, '?')
        return 'Idle' in str(grbl_out)

def homingCycle():
    cycle = ['?', 'G91', 'G38.2Z2500F2000', 'G1Z-20F1000',
             'G38.2X-560Z-560F300', 'G1X10Z10F300', 'G10L20P1X0Y0Z0',
            'G90', 'G1Z-1650F3000']
    for c in cycle:
        sendMotionCommand(c)
    motionFinished(wait=True)

def recordSensors(numSamples):
    sensorSerial.flushInput()
    samplesRead = 0
    df = pd.DataFrame(columns=['time', 'loadCell', 'level'], dtype='float')
    while(samplesRead < numSamples):
        samplesRead += 1
        data = np.array(sensorSerial.readline().decode().rstrip().split(','))
        data = data.astype(np.float)
        df.loc[len(df)] = data
    df.set_index(['time'])
    return df

def meanLoadCell(numSamples):
    data = recordSensors(numSamples)
    return data['loadCell'].mean()

def meanWeight(numSamples):
    data = recordSensors(numSamples)
    mean = data['loadCell'].mean()
    return loadCellToGrams(mean)

def calibrateLoadCell(numWeights=1):
    grams = []
    values = []
    print("Load Cell Calibration")
    print("Leave the specimen holder empty")
    input("Hit enter when the specimen holder is stationary")
    print("Wait for 5 seconds...")
    grams.append(0.0)
    values.append(meanLoadCell(80*5))
    while(numWeights > 0):
        numWeights -= 1
        while(True):
            try:
                calWeight = input("Mass of the calibration weight in kg: ").rstrip().replace(',', '.')
                grams.append(float(calWeight)*1000.0)
                break
            except:
                print("Insuitable mass value")

        print("Add the calibratrion weight to the specimen holder")
        input("Hit enter when the specimen holder is stationary")
        print("Wait for 5 seconds...")
        values.append(meanLoadCellmeanWeight(80*5))
    calibrationConstants = np.polyfit(values, grams, 1)
    print("Calibration: y = {:.2f}x+{:.2f}".format(calibrationConstants[0], calibrationConstants[1]))
    save = input("Save calibration (y/n)")
    if('y' in save.lower()):
        np.savetxt('loadCellCalibration.csv', calibrationConstants, delimiter=',')
        print('Calibration saved')
    else:
        print('Calibration cancelled')
        

In [154]:
initMotionControl()
finished = motionFinished(wait=True)
homingCycle()
closeSerial(mcSerial)

In [149]:
initSensors()
#data = recordSensors(10)
mw = meanWeight(100)
#calibrateLoadCell(numWeights = 2)
closeSerial(sensorSerial)
print(mw)

5.847726379830419


In [12]:
import Volumeter

volumeter = Volumeter.Volumeter()

/dev/ttyUSB1


NameError: name 'readLoadCellCalibration' is not defined

In [ ]:
os.system('sudo shutdown.sh')